# 4.1 Bestimmen Sie, welche Felder Ihrer Daten für Ihr Modell besonders aussagekräftig sind.

Für die Spalte "high", ist die Spalte "volume" am wichtigsten. Denn alleine die anderen Spalten sind sonst nutzlos. Mit der Spalte "volume" allerdings kann dann eine sinnvolle Berechnung durchgeführt werden, wobei die "nutzlosen Spalten" mit der Spalte "volume" verrechnet werden können.

# 4.2 Wählen Sie eine geeignete Messmetrik für Ihr Modell und berechnen Sie sie.

In [ ]:
%pip install -U scikit-learn

import pandas as pd
import os

bitcoin = os.path.join("Datenset_LB_M259 - Demo.csv")
Bitcoin = pd.read_csv(bitcoin)
Bitcoin.head()

# x = features
X = Bitcoin[["low", "open", "close", "volume"]]

# y = targets
y = Bitcoin[["high"]]
X.head()

y=y.astype("int")
# Dieser Codeabschnitt gibt die Anzahl zeilen zurück die NaN enthalten.
len(X[X.isna().any(axis=1)])
from sklearn.impute import SimpleImputer
import numpy as np
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
si.fit(X)
X_ = si.transform(X)
X = pd.DataFrame(X_, columns=X.columns)
len(X[X.isna().any(axis=1)])
# Da die Ausgabe gleich Null ist, muss ich nichts weiteres unternehmen.
# Ausserdem habe ich keine NaN Felder. Deshalb muss ich auch da nichts unternehmen.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size = 0.2, random_state = 42
)
len(X_train), len(X_test)

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

algorithms = {
    # "Nearest Neighbors" : KNeighborsClassifier(3),
    # "Stochastic Gradient Descent" : SGDClassifier(),
    # "Linear SVM" : SVC(kernel="linear", C=0.025),
    # "Decision Tree" : DecisionTreeClassifier(max_depth=5),
    # "Random Forest" : RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    # "Neural Net" : MLPClassifier(alpha=1, max_iter=1000),
    "Naive Bayes" : GaussianNB(),
    # "LDA" : LinearDiscriminantAnalysis(),
}


for name, algorithm in algorithms.items():
    algorithm.fit(X_train, y_train.values.ravel())

    score = algorithm.score(X_test, y_test)
    print(name, round(score,2))

    #Medell speichern
import joblib
joblib.dump(algorithms["Naive Bayes"], 'bitcoin_Naive_Bayes.joblib')
best_model = joblib.load('bitcoin_Naive_Bayes.joblib')
best_model

leo = pd.DataFrame([[61868.81, 61888.18, 61919.98, 2.94607889]], columns=X_train.columns)
leo.head()

Sahra = pd.DataFrame([[61882.74, 61903.16, 61882.74, 2.94235679]], columns=X_train.columns)
Sahra.head()

Michelle = pd.DataFrame([[61854.82, 61854.82, 61903.15, 3.0478475]], columns=X_train.columns)
Michelle.head()

predictions1 = best_model.predict(pd.concat([leo]))
for person, prediction1 in zip(["Leo"], predictions1):
    print("Erwartet: 61920, Berechnet:", prediction1)

predictions2 = best_model.predict(pd.concat([Sahra]))
for person, prediction2 in zip(["Sahra"], predictions2):
    print("Erwartet: 61912.67, Berechnet:", prediction2)

predictions3 = best_model.predict(pd.concat([Michelle]))
for person, prediction3 in zip(["Michelle"], predictions3):
    print("Erwartet: 61907.96, Berechnet:", prediction3)

In [ ]:
from sklearn.metrics import classification_report

best_model.fit(X_train, y_train)
predictions1 = best_model.predict(X_test)

print(classification_report(predictions1, y_test))

# 4.3 Wählen Sie geeignete Bedingungen und erstellen Sie eine Wahrheitsmatrix für Ihr Modell. Berechnen Sie darüber hinaus Sensitivität und Spezifizität.

In [ ]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression

# instantiate model
logreg = LogisticRegression()

# fit model
logreg.fit(X_train, y_train)

# make class predictions for the testing set
y_pred_class = logreg.predict(X_test)

# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

# examine the class distribution of the testing set (using a Pandas Series method)
y_test.value_counts()

# calculate the percentage of ones
# because y_test only contains ones and zeros, we can simply calculate the mean = percentage of ones
y_test.mean()

# calculate the percentage of zeros
1 - y_test.mean()

# calculate null accuracy in a single line of code
# only for binary classification problems coded as 0/1
max(y_test.mean(), 1 - y_test.mean())

# calculate null accuracy (for multi-class classification problems)
y_test.value_counts().head(1) / len(y_test)

# print the first 25 true and predicted responses
print('True:', y_test.values[0:25])
print('False:', y_pred_class[0:25])

# IMPORTANT: first argument is true values, second argument is predicted values
# this produces a 2x2 numpy array (matrix)
print(metrics.confusion_matrix(y_test, y_pred_class))

# print the first 25 true and predicted responses
print('True', y_test.values[0:25])
print('Pred', y_pred_class[0:25])

# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred_class)
print(confusion)
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

# use float to perform true division, not integer division
print((TP + TN) / float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_test, y_pred_class))

classification_error = (FP + FN) / float(TP + TN + FP + FN)

print(classification_error)
print(1 - metrics.accuracy_score(y_test, y_pred_class))

sensitivity = TP / float(FN + TP)

print(sensitivity)
print(metrics.recall_score(y_test, y_pred_class))

specificity = TN / (TN + FP)

print(specificity)

false_positive_rate = FP / float(TN + FP)

print(false_positive_rate)
print(1 - specificity)

Fazit: Für Dieses Datenset ist es nicht möglich die Teilaufgabe 4.3 durchzuführen. Es wird dafür eine Spalte in y benötigt, die nur aus Nullen und Einsen besteht. Oder das man eine Spalte umwandelt. Dies ist allerdings nicht möglich, da mein Datenset keine Spalten hat, wo es logisch wäre die spalten in Nullen und Einsen zu konvertieren.

# 4.4 Fassen Sie in 50 bis 100 Wörtern zusammen, wie gut Ihr Modell funktioniert, und stellen Sie Hypothesen auf, warum.

Für Die Aufgaben 1, 2 und 3 ist mein Datenset sehr gut geeignet. Dies liegt wahrscheindlich daran, dasss die Spalten alle einen Zusammenhang haben und nur aus Zahlen bestehen. Die einzige Schwierigkeit ist die Spalte "time". Diese habe ich bei der Berechnung einfach nicht beachtet.
Für die Teilaufgabe 4 ist mein Datenset nicht so geeignet. Denn diese erfordert eine Spalte in y, die nur aus Nullen und Einsen besteht. Dies war allerdings nicht möglich.